# Prova di esame (laboratorio)

La prova avrà una durata massima di due ore e porterà a una valutazione massima di 21 punti. Il voto finale si otterrà per somma del punteggio del test di teoria e della prova di laboratorio. 

La prova consiste in un insieme di quiz proposti attraverso un Jupyter Notebook (non sono ammessi altri strumenti software) da svolgere sui computer del laboratorio (non sono ammessi altri strumenti hardware). Durante la prova si può consultare il libro di testo, le slide usate a lezione, un manuale python (approvato dal docente) oppure le guide in linea suggerite da Jupyter Notebook. Non sono ammessi altri strumenti di consultazione. 

**Prima della prova**

Specificare la informazioni richieste nelle celle sottostanti e salvare il notebook con il proprio nome.

**Durante la prova**

Nelle celle markdown, cancellare il testo "YOUR ANSWER HERE" con la propria risposta. Nelle celle codice, rimuovere la linea di codice
```python
raise NotImplementedError()
```
e sostituirla con la propria implementazione.
Dopo ogni cella codice che deve contenere la risposta al quesito, esiste una cella codice con delle *asserzioni*, ossia delle istruzioni che verificano la correttezza della propria soluzione. (Queste asserzioni saranno usate per valutare la correttezza della risposta data e assegnare un punteggio.) E' possibile eseguire queste celle per verificare che la soluzione proposta sia corretta.


**Dopo la prova**

Salvare il file dopo aver terminato. Consegnare l'elaborato utilizzando la piattaforma ADA. 

## Dataset

### Dati Bond Yield
#### Descrizione

Medie mensili del rendimento di un'obbligazione aziendale con rating AAA di Moody's (in percentuale/anno).  

#### Formato

File CSV con 60 osservazioni di 2 variabili (più l'indice di riga):
- **time** rappresenta il mese e l'anno secondo il formato frazionario (p.e. 1990.0 rappresenta "gennaio 1990", 1990.5 rappresenta "luglio 1990", 1990.75 rappresenta "ottobre 1990", etc.)
- **value** rappresenta il rendimento percentuale (p.e. 8.99 rappresenta "8.99%")


## Domande

### Caricamento dati

Definire un dataframe di nome `dataset` e popolarla con le righe del file `BondYield.csv`. Utilizzate la prima colonna come indice di riga (utilizzare il parametro `index_col`)

In [1]:
import pandas as pd

dataset = pd.read_csv('BondYield.csv' , index_col = 0)

dataset.head()

,time,value
1,1990.000000,8.99
2,1990.083333,9.72
3,1990.166667,9.37
4,1990.250000,9.46
5,1990.333333,9.47


In [2]:
assert dataset.shape[0] == 60   # 60 osservazioni

In [3]:
assert dataset.shape[1] >= 2   # almeno 2 colonne

In [4]:
assert dataset.shape[1] == 2   # esattamente 2 colonne se la prima è usata come indice di riga

### Funzioni

Definire una funzione di nome `date_fromFractional` che riceve un parametro di nome `frac_date` (assunto float) e restituisce una tupla `(month,year)` che corrisponde al mese e all'anno rappresentato da `frac_date`. Per esempio:
- `date_fromFractional(1990.5) == (7, 1990)`
- `date_fromFractional(1990.75) == (10, 1990)`

**Nota** la funzione deve restituire risultati corretti per le date presenti nel dataset

In [5]:
import datetime as dt
from math import ceil

def date_fromFractional(frac_date):
 
    date = dt.date(int(frac_date) , 1 , 1) + dt.timedelta(days = ceil(365 * (frac_date % 1)))
    
    return (date.month , date.year)


print(date_fromFractional(1990.75))

(10, 1990)


In [6]:
assert date_fromFractional(1989.0) == (1, 1989)   # mese di gennaio corretto

In [7]:
assert date_fromFractional(1990.5) == (7, 1990)   # mese di luglio corretto

In [8]:
assert date_fromFractional(dataset.time.iat[1]) == (2, 1990) # febbraio 1990 come nel dataset

In [9]:
assert all(date_fromFractional(1990+i/12) == (i+1, 1990) for i in range(12)) # tutti i mesi alla precisione di macchina

### Manipolazione dataframe

Sostituire la colonna `time` di `dataset` con due colonne, denominate `month` e `year` che conterranno, rispettivamente, mese e anno restituite dalla funzione `date_fromFractional` applicata al valore di `time`.

**Attenzione** Gli indici del dataset iniziano da 1, non da 0. Si suggerisce di costruire prima due Series contenenti il mese e l'anno, con gli stessi indici del dataset (l'indice del dataset si ottiene con l'attributo `dataset.index`) e poi unire le due Series al dataset. Infine si può rimuovere la colonna `time`.

*Nota:* Se la funzione `date_fromFractional` non è stata correttamente implementata, inserire a mano i valori di mese e anno.

In [10]:
time = 'time'
dataset[['month', 'year']] = [(date_fromFractional(time)[0] , date_fromFractional(time)[1]) for time in dataset.pop(time)]

dataset.head()

,value,month,year
1,8.99,1,1990
2,9.72,2,1990
3,9.37,3,1990
4,9.46,4,1990
5,9.47,5,1990


In [11]:
assert not dataset.month.empty   # è presente la colonna month

In [12]:
assert not dataset.year.empty    # è presente la colonna year

In [13]:
assert (dataset.month.iat[0], dataset.year.iat[0]) == (1, 1990) # le nuove colonne hanno valori corretti

In [14]:
assert time not in dataset.columns # la colonna `time` è stata rimossa

In [15]:
assert 0 not in dataset.index  # l'indice 0 non è presente nel dataset.

### Dizionari

Definire un dizionario denominato `yearly_stats` che associa a ciascun anno presente nel dataset un altro dizionario con le voci `'min'` e `'max'` che contengano rispettivamente, il minimo e il massimo valore della colonna `value` del dataset, limitatamente all'anno della voce principale. Per esempio, `yearly_stats` conterrà la voce `1990` il cui valore è un dizionario con la voce `'min'` con valore `8.99` e la voce `'max'` con valore `9.72`, corrispondenti ai rendimenti minimi e massimi per l'anno 1990 registrati nel dataset.

**Nota** Assicurarsi che gli anni siano memorizzati come voci del dizionario siano di tipo `int`.

In [16]:
yearly_stats = dict(
    (elem , {'min' : dataset[dataset.year == elem].value.min() , 'max' : dataset[dataset.year == elem].value.max()}) 
    for elem in dataset.year
)
    
yearly_stats

{1990: {'min': 8.99, 'max': 9.72},
 1991: {'min': 8.31, 'max': 9.04},
 1992: {'min': 7.92, 'max': 8.35},
 1993: {'min': 6.66, 'max': 7.91},
 1994: {'min': 6.92, 'max': 8.68}}

In [17]:
assert all(y in yearly_stats for y in range(1990, 1994+1))  # tutti gli anni nel dizionario

In [18]:
assert all(all(k in yearly_stats[y] for k in ['min','max']) for y in yearly_stats) # struttura corretta del dizionario

In [19]:
assert all(yearly_stats[y]['min'] <= yearly_stats[y]['max'] for y in yearly_stats) # contenuto valido

In [20]:
assert all(type(y) == int for y in yearly_stats)   # (Non valutato) Attenzione al tipo della chiave del dizionario

### JSON

Salvare il dizionario `yearly_stats` in un file denominato `yearly_stats.json` nel formato JSON.

In [21]:
import json

with open("yearly_stats.json", "w") as yearly_st:
    json.dump(yearly_stats , yearly_st)

In [22]:
with open('yearly_stats.json', 'r') as jsonfile:
    test_dict = json.load(jsonfile)
    

### Comprensioni di lista

Estrarre la colonna `value` del dataset e memorizzarla in una lista denominata `yields`. Da questa lista definire un'altra lista denominata `yields_diff` il cui valore in posizione `i` è `'+'` se la differenza tra i valori in posizione `i+1` e `i` in `yields` è positiva, altrimenti sarà `'-'`. Utilizzare le comprensioni di lista per calcolare il risultato. Prestare attenzione alla lunghezza della lista risultante.

In [23]:
yields = list(dataset.value)
yields_diff = ['+' if yields[i] - yields[i - 1] > 0 else '-' for i in range(1 , len(yields))]

yields_diff[0:10]

['+', '-', '+', '+', '-', '-', '+', '+', '-', '-']

In [24]:
assert yields_diff[0:10] == ['+', '-', '+', '+', '-', '-', '+', '+', '-', '-'] # Contenuto corretto

In [25]:
assert len(yields_diff) == len(yields)-1 # Lunghezza corretta

### Espressioni regolari

Convertire `yields_diff` in una stringa e memorizzare il risultato in una variabile denominata `yields_diff_str`.

In [26]:
yields_diff_str = ''.join(elem for elem in yields_diff)

yields_diff_str

'+-++--++-----+--+-------++------++----------++--++++-+-+++-'

In [27]:
assert all(c in yields_diff_str for c in '+-') and type(yields_diff_str) == str # yields_diff_str è una stringa corretta

Utilizzando le espressioni regolari, estrarre tutte le sottostringhe contenenti almeno 3 caratteri uguali. Memorizzare le sottostringhe in una lista denominata `subs`.

In [28]:
import re

subs = re.findall(r'[+]{3,}|[-]{3,}' , yields_diff_str)

subs

['-----', '-------', '------', '----------', '++++', '+++']

In [29]:
assert all(len(s)>=3 for s in subs)   # Tutte le sottostringhe hanno almeno tre caratteri

In [30]:
assert all(len(set(s)) == 1 for s in subs) # Ogni sottostringa ha lo stesso carattere replicato più volte.